# Collecting Our Economic Status Data

## Import Libraries

In [2]:
import requests
import os
import io
import boto3

## Starting our AWS Session

In [3]:
session = boto3.Session(profile_name='personal-onemap')
s3 = session.resource('s3')

## Import API Authorisation Key

In [4]:
# For OneMap Account Key
from dotenv import load_dotenv
load_dotenv()
account_key = os.getenv('ONEMAP_KEY')
headers = {'Authorization': f'Bearer {account_key}'}

## Count number of areas in Singapore

In [5]:
api_url = "https://www.onemap.gov.sg/api/public/popapi/getPlanningareaNames?year=2019" # Latest info from 2019

response = requests.get(api_url, headers=headers, timeout=1)
data = response.json()

# Count number of area codes
area_count = sum(1 for item in data if "pln_area_n" in item)
print(f'Number of areas: {area_count}')

# Save our area codes into List
area_list = [item['pln_area_n'] for item in data if "pln_area_n" in item]
print(area_list)


Number of areas: 55
['BEDOK', 'BUKIT TIMAH', 'BUKIT BATOK', 'BUKIT MERAH', 'CENTRAL WATER CATCHMENT', 'DOWNTOWN CORE', 'CHANGI', 'CHANGI BAY', 'LIM CHU KANG', 'BOON LAY', 'WESTERN WATER CATCHMENT', 'WOODLANDS', 'MARINE PARADE', 'NEWTON', 'NORTH-EASTERN ISLANDS', 'ORCHARD', 'PASIR RIS', 'PIONEER', 'PUNGGOL', 'QUEENSTOWN', 'SEMBAWANG', 'SIMPANG', 'TAMPINES', 'TANGLIN', 'TUAS', 'WESTERN ISLANDS', 'SOUTHERN ISLANDS', 'BUKIT PANJANG', 'BISHAN', 'ANG MO KIO', 'GEYLANG', 'STRAITS VIEW', 'JURONG EAST', 'HOUGANG', 'JURONG WEST', 'CHOA CHU KANG', 'KALLANG', 'MANDAI', 'TENGAH', 'MARINA EAST', 'MARINA SOUTH', 'MUSEUM', 'NOVENA', 'OUTRAM', 'PAYA LEBAR', 'RIVER VALLEY', 'ROCHOR', 'SELETAR', 'SENGKANG', 'SERANGOON', 'CLEMENTI', 'TOA PAYOH', 'SINGAPORE RIVER', 'SUNGEI KADUT', 'YISHUN']


## Test API Output

In [6]:
response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea=Bedok&year=2000&gender=male", headers=headers, timeout=1)
data = response.json()

print(response)
print(data)

<Response [200]>
[{'planning_area': 'Bedok', 'employed': 71767, 'unemployed': 4653, 'inactive': 26300, 'year': 2000, 'gender': 'Male'}]


## Retriving Economic Status in 2000 for Males

In [ ]:
year = '2000'  
gender = 'male'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")


## Retriving Economic Status in 2000 for Females

In [ ]:
year = '2000'  
gender = 'female'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1.2)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")


## Retriving Economic Status in 2010 for Males

In [ ]:
year = '2010'  
gender = 'male'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")


## Retriving Economic Status in 2010 for Females

In [ ]:
year = '2010'  
gender = 'female'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")


## Retriving Economic Status in 2015 for Males

In [ ]:
year = '2015'  
gender = 'male'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")


## Retriving Economic Status in 2015 for Females

In [ ]:
year = '2015'  
gender = 'female'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")


## Retriving Economic Status in 2020 for Males

In [ ]:
year = '2020'  
gender = 'male'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")


## Retriving Economic Status in 2020 for Females

In [ ]:
year = '2020'  
gender = 'female'  


csv_file = io.StringIO()
csv_file.write('planning_area,employed,unemployed,inactive,year,gender\n')

for area in area_list:
    response = requests.get(f"https://www.onemap.gov.sg/api/public/popapi/getEconomicStatus?planningArea={area}&year={year}&gender={gender}", headers=headers, timeout=1)
    data = response.json()
    
    if response.status_code == 200:
        # Check for the specific case of "No Data Available!" (When Area has not been created since 2019)
        if 'Result' in data and data['Result'] == "No Data Available!":
            print(f"{area} has no results.")
        elif data:
            for item in data:
                csv_file.write(f"{item['planning_area']},{item['employed']},{item['unemployed']},{item['inactive']},{item['year']},{item['gender']}\n")
            print(f"Success: {area} has been logged.")
    else:
        print(f"Failed to process {area}.")

print("All areas processed.")

# Move back to start of the file
csv_file.seek(0)

# Upload to S3
s3.Object("onemap-s3", f'Economic_Status_Data/{year}/' + f'economic_status_by_area_{year}_{gender}.csv').put(Body=csv_file.getvalue(), ContentType='text/csv')
csv_file.close()

print(f"Successfully Uploaded to S3")
